In [6]:
#### Import packages needed (and some not needed....)

import Bio

import openpyxl
from Bio import SeqIO
import os,sys,re

import pandas as pd
print(pd.__name__, pd.__version__)

import numpy as np
print(np.__name__, np.__version__)

import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__name__, matplotlib.__version__)
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator

import scipy.stats as stats
import scipy
print(scipy.__name__, scipy.__version__)

import seaborn as sns
print(sns.__name__, sns.__version__)

from collections import Counter

import scipy as sp
import math
from scipy.stats import shapiro 
from scipy.stats import lognorm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_rel

pandas 1.5.2
numpy 1.23.5
matplotlib 3.4.1
scipy 1.10.0
seaborn 0.12.2


In [7]:
df_none = pd.read_csv("../Nothing/Select_coeff_p_value_aa_No antifungal_NovaSeq_winter2023_albicans_2023-12-07.csv", header=0)
df_fluco = pd.read_csv("../Fluconazole/Select_coeff_p_value_aa_Fluconazole_NovaSeq_winter2023_albicans_2023-12-06.csv", header=0)
df_itra = pd.read_csv("../Itraconazole/Select_coeff_p_value_aa_Itraconazole_NovaSeq_winter2023_albicans_2023-12-06.csv", header=0)
df_vorico = pd.read_csv("../Voriconazole/Select_coeff_p_value_aa_Voriconazole_NovaSeq_winter2023_albicans_2023-12-06.csv", header=0)
df_posaco = pd.read_csv("../Posaconazole/Select_coeff_p_value_aa_Posaconazole_NovaSeq_winter2023_albicans_2023-12-06.csv", header=0)
df_isa = pd.read_csv("../Isavuconazole/Select_coeff_p_value_aa_Isavuconazole_NovaSeq_winter2023_albicans_2023-12-06.csv", header=0)
df_clotri = pd.read_csv("../Clotrimazole/Select_coeff_p_value_aa_Clotrimazole_NovaSeq_winter2023_albicans_2023-12-06.csv", header=0)

In [22]:
#open df with wt positions

WT_annot = pd.read_excel('../wt_aa_annot.xlsx', header=0)

aa_wt1 = pd.melt(WT_annot, id_vars=['aa'])

#keep only WT rows

aa_wt1 = aa_wt1[aa_wt1['value'] == 'o'].reset_index(drop=True)

#make a dictionnary of position + wt aa

aa_dict = dict(zip(aa_wt1['variable'], aa_wt1['aa']))

In [8]:
# List of dataframes
df_list = [df_none, df_fluco, df_itra, df_vorico, df_posaco, df_isa, df_clotri]

# Iterate over each dataframe
for df in df_list:
    # Create a new column 'result' based on conditions
    df['result'] = df.apply(lambda row: 'advantageous' if (row['adjusted_p_value'] < 0.01 and row['select_coef'] > 0)
                       else 'deleterious' if (row['adjusted_p_value'] < 0.01 and row['select_coef'] < 0)
                       else 'wt like', axis=1)

In [23]:
All_long = pd.concat([df_none, df_fluco,df_itra,df_vorico,df_posaco,df_isa, df_clotri],axis=0)

All_long['aa_WT'] = All_long['position'].map(aa_dict)
All_long.to_csv(f'select_coef_all_conditions_CB_2023-10-16.csv')  
All_long

,Unnamed: 0,position,Antifungal,aa,select_coef,p_value,adjusted_p_value,variant,result,aa_WT
0,0,55,None,*,-0.528211,8.240903e-210,8.053801e-209,55*,deleterious,I
1,1,55,None,A,-0.004103,7.384538e-01,8.869498e-01,55A,wt like,I
2,2,55,None,C,-0.009924,3.218775e-01,5.661675e-01,55C,wt like,I
3,3,55,None,D,-0.057636,9.785407e-11,4.824697e-10,55D,deleterious,I
4,4,55,None,E,-0.014590,3.389770e-02,9.943499e-02,55E,wt like,I
...,...,...,...,...,...,...,...,...,...,...
4017,4017,511,Clotri,S,0.075024,1.990360e-04,2.296708e-04,511S,advantageous,L
4018,4018,511,Clotri,T,-0.135075,1.193329e-12,1.606904e-12,511T,deleterious,L
4019,4019,511,Clotri,V,0.058365,9.611839e-03,1.057220e-02,511V,wt like,L
4020,4020,511,Clotri,W,0.093816,1.020725e-03,1.153510e-03,511W,advantageous,L


In [10]:
All_long = All_long[All_long["Antifungal"] != "None"]

In [11]:
df_dele = All_long.groupby("result")["select_coef"].median().reset_index()
df_dele

,result,select_coef
0,advantageous,0.144825
1,deleterious,-0.399891
2,wt like,0.001885


In [12]:
df_pos_med = All_long.groupby("position")["select_coef"].median().reset_index()
positions_to_remove = [432, 434, 436]
df_pos_med = df_pos_med[~df_pos_med["position"].isin(positions_to_remove)]
df_pos_med["position"] = ":" + df_pos_med["position"].astype(str)

In [13]:
df_pos_max = All_long.groupby("position")["select_coef"].max().reset_index()
positions_to_remove = [432, 434, 436]
df_pos_max = df_pos_max[~df_pos_max["position"].isin(positions_to_remove)]
df_pos_max["position"] = ":" + df_pos_max["position"].astype(str)

In [16]:
##Make the necessary file to color CaErg11 structure on Chimera X from selection coefficient values

#Median select coef azoles

pd.set_option('mode.chained_assignment', None) # Disable settingWithCopy warning

df_pos_med['empty_col'] = ''
df_chi = df_pos_med[['empty_col','position', 'select_coef']]
            
custom_header = f'#\nattribute: scoeff\nmatch mode: 1-to-1\nrecipient: residues\n'
with open(f'defattr_files/med_scoeff_attr.txt', 'w') as fp:
    fp.write(custom_header)

df_chi.to_csv(f'defattr_files/med_scoeff_attr.txt', sep='\t', header=None, index=None, mode='a')
 # Palette below is specified manually because converting seaborn palette to discrete colors excludes extreme values
 # Was done with online hexcolor picker so not 100% accurate, also ChimeraX still does not manage to re-create cmap as expected...
cmd = f'color byattribute scoeff range -0.2, 0.2 palette #B95439:#B95438:#B95437:#FFFFFF:#386461'
print(cmd)

color byattribute scoeff range -0.2, 0.2 palette #B95439:#B95438:#B95437:#FFFFFF:#386461


In [17]:
#Max select coef azoles

pd.set_option('mode.chained_assignment', None) # Disable settingWithCopy warning

df_pos_max['empty_col'] = ''
df_chi = df_pos_max[['empty_col','position', 'select_coef']]
            
custom_header = f'#\nattribute: scoeff\nmatch mode: 1-to-1\nrecipient: residues\n'
with open(f'defattr_files/max_scoeff_attr.txt', 'w') as fp:
    fp.write(custom_header)

df_chi.to_csv(f'defattr_files/max_scoeff_attr.txt', sep='\t', header=None, index=None, mode='a')
 # Palette below is specified manually because converting seaborn palette to discrete colors excludes extreme values
 # Was done with online hexcolor picker so not 100% accurate, also ChimeraX still does not manage to re-create cmap as expected...
cmd = f'color byattribute scoeff range 0, 1 palette #B95439:#B95438:#B95437:#FFFFFF:#386461'
print(cmd)

##print(cmd) gives the command to enter in chimera command line

color byattribute scoeff range 0, 1 palette #B95439:#B95438:#B95437:#FFFFFF:#386461


In [20]:
df_none_med = df_none.groupby("position")["select_coef"].median().reset_index()
positions_to_remove = [432, 434, 436]
df_none_med = df_none_med[~df_none_med["position"].isin(positions_to_remove)]
df_none_med["position"] = ":" + df_none_med["position"].astype(str)

In [21]:
##Med select coef control condition

pd.set_option('mode.chained_assignment', None) # Disable settingWithCopy warning

df_none_med['empty_col'] = ''
df_chi = df_none_med[['empty_col','position', 'select_coef']]
            
custom_header = f'#\nattribute: scoeff\nmatch mode: 1-to-1\nrecipient: residues\n'
with open(f'defattr_files/med_none_scoeff_attr.txt', 'w') as fp:
    fp.write(custom_header)

df_chi.to_csv(f'defattr_files/med_none_scoeff_attr.txt', sep='\t', header=None, index=None, mode='a')
 # Palette below is specified manually because converting seaborn palette to discrete colors excludes extreme values
 # Was done with online hexcolor picker so not 100% accurate, also ChimeraX still does not manage to re-create cmap as expected...
cmd = f'color byattribute scoeff range -0.2, 0.2 palette #B95439:#B95438:#B95437:#FFFFFF:#386461'
print(cmd)

color byattribute scoeff range -0.2, 0.2 palette #B95439:#B95438:#B95437:#FFFFFF:#386461
